# 当代人工智能实验一：文本分类
## ——TF-IDF & Logistic Regression

### 一. 引入必要模块
numpy将用于数据的处理。
CountVectorizer, TfidfVectorizer, TfidfTransformer是在利用TF-IDF方法将文本映射为向量。
LogisticRegression用于进行逻辑回归。
train_test_split用于进行训练集与验证集的划分。

In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import time

### 二. 读取训练与测试数据
每条训练集数据都包含“文本”与给定的“标签”。每条测试集数据都有编号与“文本”。

In [2]:
# 打开并读取训练数据文档
f_train = open('train_data.txt')
train_text = f_train.read()
# print(train_text)
# 观察数据特征，确定数据文档中的每一项均由一个回车分割，故采取切片
train_text = train_text.split("\n")
# 创造两个数组，存储训练数据
# labelList 存储每个数据的标签
# rawList 存储每个数据的文本内容
labelList = []
rawList = []
for i in range(len(train_text)-1):
    train_text[i] = eval(train_text[i])
    labelList.append(train_text[i]["label"])
    rawList.append(train_text[i]["raw"])
labelList = np.array(labelList)
rawList = np.array(rawList)

In [3]:
# 打开并读取测试数据文档
f_test = open('test.txt')
result = f_test.read()
# print(test_text)
# 观察数据特征，确定数据文档中的每一项均由一个回车分割，故采取切片
result = result.split("\n")
# 去除第一行
result.pop(0)
# 测试集的大小
TEST_LENGTH = 2000
result_id = list(range(TEST_LENGTH))
result_text = []
for i in range(TEST_LENGTH):
    comma_index = result[i].find(",")
    if comma_index != -1:
        result_text.append(result[i][comma_index+2:])
    else:
        print("ERROR: COMMA NOT FOUND")
result_text = np.array(result_text)
# result_text

In [4]:
ALL = np.append(rawList, result_text)
# 记录数据的个数
LENGTH_TRAIN = len(rawList)
LENGTH_ALL = len(ALL)
# ALL

保存这些向量，可以用于未来直接使用。

In [5]:
np.save('rawList.npy', rawList)
np.save('labelList.npy', labelList)

### 三. 文本转化为向量
这里使用TF-IDF方法。

In [6]:
# 初始化训练参数
# analyzer为word时，表示以单词为单位计算TF-IDF值
# stop_words为english时，表示去除英语中的停用词，避免缺乏实际意义的计算
# use_idf为True时，表示要计算idf值
# smooth_idf为True时，表示要在文档频率上加一来平滑idf，避免分母为0
# norm为None时，表示输出结果不需要任何标准化或归一化，为l2时，说明输出结果进行了归一化
start_time = time.time()
tv1 = TfidfVectorizer(analyzer="word", stop_words="english", use_idf=True, smooth_idf=True, norm="l2")
tv1_fit = tv1.fit_transform(ALL)
# 得到每一个文本对应的TF-IDF向量
vsm_matrix = tv1_fit.toarray()
# vsm_matrix
textList = []
result_list = []
for i in range(LENGTH_TRAIN):
    textList.append(vsm_matrix[i])
textList = np.array(textList)
np.save('textVectorList.npy', textList)
for i in range(LENGTH_TRAIN, LENGTH_ALL):
    result_list.append(vsm_matrix[i])
result_list = np.array(result_list)
np.save('result_list.npy', result_list)
end_time = time.time()
print("运行时间为:", end_time - start_time)

result_list

运行时间为: 11.171382904052734


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### 四. 利用训练集数据训练多分类逻辑回归模型
使用sklearn中的LogisticRegression模块。
在这里，我们通过蒙特卡洛交叉验证来验证模型的正确性。

In [7]:
# 划分训练集与测试集，这里选取12.5%的数据作为测试集，剩余数据作为训练集。
# random_state数值是不同会让训练集与测试集不同，若写为None则每次都随机生成。
accuracyTotal = 0
precisionTotal = 0
recallTotal = 0
f1Total = 0
LOOP_NUMBER = 5
target_names = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

start_time = time.time()
for loop in range(LOOP_NUMBER):
    text_train, text_test, label_train, label_test = train_test_split(textList, labelList, test_size=0.125, random_state=None)
    model = LogisticRegression(max_iter=1000)
    model.fit(text_train, label_train)
    accuracy = model.score(text_test, label_test)

    y_pred = model.predict(text_test)
    classification_rep = classification_report(label_test, y_pred, target_names=target_names, output_dict=True)

    # 提取相应的指标值
    precision = classification_rep['weighted avg']['precision']
    recall = classification_rep['weighted avg']['recall']
    f1 = classification_rep['weighted avg']['f1-score']

    accuracyTotal += accuracy
    precisionTotal += precision
    recallTotal += recall
    f1Total += f1

end_time = time.time()
# 计算平均值
runtime = (end_time - start_time) / LOOP_NUMBER
accuracy_avg = accuracyTotal / LOOP_NUMBER
precision_avg = precisionTotal / LOOP_NUMBER
recall_avg = recallTotal / LOOP_NUMBER
f1_avg = f1Total / LOOP_NUMBER

print("平均运行时间为：", runtime)
print("模型准确率：", accuracy_avg)
print("模型精确度：", precision_avg)
print("模型召回率：", recall_avg)
print("模型F1-score：", f1_avg)

平均运行时间为： 69.24085001945495
模型准确率： 0.9480000000000001
模型精确度： 0.9495307501890655
模型召回率： 0.9480000000000001
模型F1-score： 0.9479776915998805


这说明模型准确率较高，可以投入使用。故我们将全部数据投入到模型中进行训练。

In [19]:
model = LogisticRegression(max_iter=1000)
model.fit(textList, labelList)

LogisticRegression(max_iter=1000)

### 五. 预测测试集结果
预测并观察最终的结果。

In [20]:
predictions = model.predict(result_list)
predictions

array([6, 1, 4, ..., 9, 6, 9])